# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=tarakan25")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [9]:
filepath = 'data/song_data'
song_files = get_files(filepath)

In [10]:
df = pd.read_json(song_files[2], lines=True, dtype=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
song_data = df[["song_id", "title", "artist_id","year", "duration"]].values.tolist()
song_data

[['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
#song_table_insert = "Insert INTO songs (song_id, title, artist_id, year,duration) values (%s,%s,%s,%s,%s)"
cur.execute(song_table_insert, song_data)
conn.commit()

# data = [(1, 'post', 'author'), (2, 'post', 'author')]
# for d in data:
#     cur.execute("INSERT into posts(id, post, author) VALUES (%s, %s, %s)", d)

In [5]:
cur.execute("Select * from songs")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
conn.commit()

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))
('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, Decimal('177.47546'))
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, Decimal('233.40363'))
('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, Decimal('209.60608'))
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, Decimal('267.7024'))
('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, Decimal('114.78159'))
('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data=df[["artist_id", "artist_name", "artist_location","artist_latitude", "artist_longitude"]].values.tolist()[0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
#artist_table_insert = "Insert INTO artists (artist_id, name, location, latitude,longitude) values (%s,%s,%s,%s,%s)"
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [6]:
def printing_cols(table_name):
    try: 
        cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME =" +table_name +";")
    except psycopg2.Error as e: 
        print("Error: select *")
        print (e)
    row = cur.fetchone()
    while row:
        print(row)
        row = cur.fetchone()
print("Table: artists2\n")

printing_cols("'artists'")

try: 
    cur.execute("SELECT * FROM artists;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Table: artists2

('artist_id',)
('name',)
('location',)
('latitude',)
('longitude',)
('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', 'NaN', 'NaN')
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', '35.14968', '-90.04892')
('ARKRRTF1187B9984DA', 'Sonora Santanera', '', 'NaN', 'NaN')
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', 'NaN', 'NaN')
('ARXR32B1187FB57099', 'Gob', '', 'NaN', 'NaN')
('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', 'NaN', 'NaN')
('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', 'NaN', 'NaN')
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', 'NaN', 'NaN')
('AR8ZCNI1187B9A069B', 'Planet P Project', '', 'NaN', 'NaN')
('ARNTLGG11E2835DDB9', 'Clp', '', 'NaN', 'NaN')
('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', '35.21962', '-80.01955')
('ARC43071187B990240', 'Wayne Watson', 'Wisner, LA', 'NaN', 'NaN')
('ARL7K851187B99ACD2', 'Andy Andy', '', 'NaN', 'NaN')
('ARHHO3O1187B989413', 'Bob Azzam', '', 'NaN', 'NaN')
('ARIK43K118

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
filepath = 'data/log_data'
log_files = get_files(filepath)

In [13]:
df = pd.read_json(log_files[2], lines=True, dtype=True)
df.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,52,None,307,1541207073796,None,
1,None,Logged In,Celeste,F,1,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

In [15]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,2018-11-03 01:05:50.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,2018-11-03 01:08:36.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,2018-11-03 01:12:26.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
5,Jimmy Eat World,Logged In,Celeste,F,5,Williams,285.83138,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Dizzy,200,2018-11-03 01:16:15.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
7,Maldita Nerea,Logged In,Anabelle,F,0,Simpson,241.16200,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,158,Supelicula,200,2018-11-03 14:17:50.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [16]:
#t = 
#t.head()

In [ ]:
time_data = [pd.Series(df['ts']).tolist(),pd.Series(df['ts']).dt.hour.tolist(), pd.Series(df['ts']).dt.day.tolist(), pd.Series(df['ts']).dt.isocalendar().week.tolist(), pd.Series(df['ts']).dt.month.tolist(), pd.Series(df['ts']).dt.year.tolist(),pd.Series(df['ts']).dt.weekday.tolist()]
column_labels = ["start_time", "hour","day","week","month","year","weekday"]

In [18]:
time_df = pd.DataFrame({column_labels[i]: time_data[i] for i in range(len(column_labels))})
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-03 01:05:50.796,1,3,44,11,2018,5
1,2018-11-03 01:08:36.796,1,3,44,11,2018,5
2,2018-11-03 01:12:26.796,1,3,44,11,2018,5
3,2018-11-03 01:16:15.796,1,3,44,11,2018,5
4,2018-11-03 14:17:50.796,14,3,44,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [20]:
def printing_cols(table_name):
    try: 
        cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME =" +table_name +";")
    except psycopg2.Error as e: 
        print("Error: select *")
        print (e)
    row = cur.fetchone()
    while row:
        print(row)
        row = cur.fetchone()
print("Table: time\n")

printing_cols("'time'")

try: 
    cur.execute("SELECT * FROM time;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

Table: time

('start_time',)
('hour',)
('day',)
('week',)
('month',)
('year',)
('weekday',)
(datetime.datetime(2018, 11, 3, 1, 5, 50, 796000), Decimal('1'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 1, 8, 36, 796000), Decimal('1'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 1, 12, 26, 796000), Decimal('1'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 1, 16, 15, 796000), Decimal('1'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 14, 17, 50, 796000), Decimal('14'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 15, 11, 20, 796000), Decimal('15'), Decimal('3'), Decimal('44'), Decimal('11'), Decimal('2018'), Decimal('5'))
(datetime.datetime(2018, 11, 3, 15, 36, 8, 796000), Decimal(

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
#df = pd.read_json(log_files[2], lines=True, dtype=True)
#df

In [22]:
user_df = df[["userId", "firstName", "lastName","gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
2,53,Celeste,Williams,F,free
3,53,Celeste,Williams,F,free
4,53,Celeste,Williams,F,free
5,53,Celeste,Williams,F,free
7,69,Anabelle,Simpson,F,free
...,...,...,...,...,...
106,49,Chloe,Cuevas,F,free
107,49,Chloe,Cuevas,F,free
108,49,Chloe,Cuevas,F,free
109,6,Cecilia,Owens,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [24]:
cur.execute("Select * from users")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()
conn.commit()

('53', 'Celeste', 'Williams', 'F', 'free')
('53', 'Celeste', 'Williams', 'F', 'free')
('53', 'Celeste', 'Williams', 'F', 'free')
('53', 'Celeste', 'Williams', 'F', 'free')
('69', 'Anabelle', 'Simpson', 'F', 'free')
('62', 'Connar', 'Moreno', 'M', 'free')
('101', 'Jayden', 'Fox', 'M', 'free')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('95', 'Sara', 'Johnson', 'F', 'paid')
('10', 'Sylvie', 'Cruz', 'F', 'free')
('95', 'Sara', 'Johnson', 'F', 'paid')
('15',

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data =(row.ts, row.userId, row.level, songid, artistid,row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    
    


In [26]:
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,2018-11-03 01:05:50.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,2018-11-03 01:08:36.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,2018-11-03 01:12:26.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
5,Jimmy Eat World,Logged In,Celeste,F,5,Williams,285.83138,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Dizzy,200,2018-11-03 01:16:15.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
7,Maldita Nerea,Logged In,Anabelle,F,0,Simpson,241.16200,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,158,Supelicula,200,2018-11-03 14:17:50.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,The Smiths,Logged In,Chloe,F,1,Cuevas,196.67546,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,195,The Boy With The Thorn In His Side,200,2018-11-03 21:14:28.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
107,Quique Gonzalez,Logged In,Chloe,F,2,Cuevas,214.20363,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,195,Cuando Eramos Reyes,200,2018-11-03 21:17:44.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
108,Muse,Logged In,Chloe,F,3,Cuevas,210.46812,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,195,Pink Ego Box,200,2018-11-03 21:21:18.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
109,Sugarland,Logged In,Cecilia,F,0,Owens,247.77098,free,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.541032e+12,5,Just Might (Make Me Believe),200,2018-11-03 23:17:02.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6


In [27]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid,row.sessionId, row.location, row.userAgent)
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()
    print(songplay_data)

(Timestamp('2018-11-03 01:05:50.796000'), '53', 'free', None, None, 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"')
(Timestamp('2018-11-03 01:08:36.796000'), '53', 'free', None, None, 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"')
(Timestamp('2018-11-03 01:12:26.796000'), '53', 'free', None, None, 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"')
(Timestamp('2018-11-03 01:16:15.796000'), '53', 'free', None, None, 52, 'Klamath Falls, OR', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"')
(Timestamp('2018-11-03 14:17:50.796000'), '69', 'free', None, None, 158, 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4)

In [28]:
Select start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

SyntaxError: invalid syntax (<ipython-input-28-98d32be10faa>, line 1)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.